<div style="float:right;border-left:1em solid transparent">
    <i>Notebooks on Programming</i>
</div>

---
# Program Derivation - Draft
**[Emil Sekerinski](http://www.cas.mcmaster.ca/~emil), McMaster University, November 2024**

---

<figure style="float:right" >
    <img style="width:9.5em" src="./img/by-nc-nd.png"/>
    <figcaption style="width:13em;font-size:80%"><a href="https://creativecommons.org/licenses/by-nc-nd/4.0/">Licensed under Creative Commons CC BY-NC-ND</a>
    </figcaption>
</figure>

## The Minimal Segment Sum

For an integer sequence `s` of length `N`, the *segment sum* `ss(i, j)` for `0 ≤ i ≤ j ≤ N` is defined as:
```algorithm
ss(i, j) = (∑ h ∈ i .. j – 1 • s(h))
```
The *minimal segment sum* is the smallest segment sum. The task is to write a program that, for integer `x`, establishes the postcondition:
```algorithm
x = (MIN i ∈ 0 .. N, j ∈ i .. N • ss(i, j))
```
For example, if `s = [1, 2, –3, 0, 2, –9, 7]`, the minimal segment sum is `–10`. Since the empty segment is allowed and has a sum of `0`, the minimal segment sum can be at most `0`. The number of combinations of `i` and `j` for which `ss(i, j)` has to be determined is `(N + 1) × (N + 2) / 2`. For each combination of `i` and `j`, in the order of `N` additions must be carried out. A direct implementation, therefore, requires time proportional to `N³`. However, a more efficient implementation is possible!

As all values of `s` need to be consulted, the choice is to iterate over all elements from `n = 0` to `N – 1`. The invariant is obtained by replacing constant `N` in the postcondition with variable `n`. Defining
```
P(n) ≡ x = (MIN i ∈ 0 .. n, j ∈ i .. n • ss(i, j))
```
the desired postcondition is `P(N)`. The resulting loop structure is:

```algorithm
"initialize x for n = 0"
{0 ≤ n ≤ N ∧ P(n)}
while n < N do
    {0 ≤ n < N ∧ P(n)}
    "modify x"
    {0 ≤ n < N ∧ P(n + 1)}
    n := n + 1
    {0 ≤ n ≤ N ∧ P(n)}
{P(N)}
```

For `"initialize x for n = 0"` we immediately get
```algorithm
n, x := 0, 0
```
from the definition of `P(n)`. For the postcondition of  `"modify x"`, we observe:
```algorithm
    P(n + 1)
=       {by definition of P}
    x = (MIN i ∈ 0 .. n + 1, j ∈ i .. n + 1 • ss(i, j))
=       {by case analysis j = n + 1 and j ≠ n + 1}
    x = min((MIN i ∈ 0 .. n, j ∈ i .. n • ss(i, j)), (MIN i ∈ 0 .. n + 1 • ss(i, n + 1)))
```
The first argument of `min` appears in the precondition `P(n)` of `"modify x"`, suggesting for `"modify x"`:
```algorithm
x := min(x, (MIN i ∈ 0 .. n + 1 • ss(i, n + 1)))
```

A direct implementation of `MIN` would make the overall runtime still proportional to `N³`, but we can improve further. The shape of the above expression suggests the introduction of a new variable, say `y`, with the invariant
```
Q(n) ≡ y = (MIN i ∈ 0 .. n • ss(i, n))
```
and update `y` in the loop. Thus, the program becomes:
```algorithm
"initialize x and y for n = 0"
{0 ≤ n ≤ N ∧ P(n) ∧ Q(n)}
while n < N do
    {0 ≤ n < N ∧ P(n) ∧ Q(n)}
    "modify y"
    {0 ≤ n < N ∧ P(n) ∧ Q(n + 1)}
    x := min(x, y)
    {0 ≤ n < N ∧ P(n + 1) ∧ Q(n + 1)}
    n := n + 1
    {0 ≤ n ≤ N ∧ P(n) ∧ Q(n)}
```

For `"initialize x and y for n = 0"` we now get:
```algorithm
n, x, y := 0, 0, 0
```
For the postcondition of  `"modify y"` we have:
```algorithm
    Q(n + 1)
=       {by definition of Q}
    y = (MIN i ∈ 0 .. n + 1 • ss(i, n + 1))
=       {by case analysis i = n + 1 and i ≠ n + 1}
    y = min((MIN i ∈ 0 .. n • ss(i, n + 1)), ss(n + 1, n + 1))
=       {by definition of ss, empty summation}
    y = min((MIN i ∈ 0 .. n • ss(i, n) + s(n)), 0)
=       {i does not appear in s(n)}
    y = min((MIN i ∈ 0 .. n • ss(i, n)) + s(n), 0)
```
The first argument of `min` appears in the precondition `Q(n)`, suggesting for `"modify y"`:
```algorithm
y := min(y + s(n), 0)
```

The resulting program, without annotations, becomes
```algorithm
n, x, y := 0, 0, 0
while n < N do
    y := min(y + s(n), 0)
    x := min(x, y)
    n := n + 1
```
or, when replacing `min` with conditional statements and exploiting the invariant `x ≤ 0`:
```algorithm
n, x, y := 0, 0, 0
while n < N do
    y := y + s(n)
    if y ≥ 0 then y := 0
    else if x > y then x := y
    n := n + 1
```
This program has a running time proportional to `N`.

Here are Python implementations of the cubic and the linear versions. A "large" array is used for timing:

In [ ]:
def minSegSumCubic(s):
    return min(sum(s[i:j]) for i in range(0, len(s) + 1) for j in range(i, len(s) + 1))

minSegSumCubic([1, 2, -3, 0, 2, -9, 7])

In [ ]:
#input is: 0, -1, 2, -3, 4, -5, 6, -7, ..., 998, -999
%time assert minSegSumCubic([(-1) ** n * n for n in range(1000)]) == - 999

In [ ]:
def minSegSumLinear(s):
    n, x, y = 0, 0, 0
    for n in range(len(s)):
        y += s[n]
        if y >= 0: y = 0
        else:
            if x > y: x = y
    return x

minSegSumLinear([1, 2, -3, 0, 2, -9, 7])

In [ ]:
%time assert minSegSumLinear([(-1) ** n * n for n in range(1000)]) == - 999

## The Concidence Count

Suppose `a` and `b` are integer sequences of length `M` and `N`, respectively, in monotonically increasing order. Writing `s[k..l)` for the subsequence of sequence `s` from `k` to `l – 1` and writing `s ≤ x` if `e ≤ x` for all elements `e ∈ s`, this is expressed as:

```algorithm
P ≡ (∀ i ∈ 0 .. M – 1 · a[0..i) < a(i)) ∧ (∀ i ∈ 0 .. N – 1 · b[0..i) < b(i))
```

The task is to find the number of integers occurring in both sequences; the program has to establish:

```algorithm
R ≡ x = #{i ∈ 0 .. M – 1, j ∈ 0 .. N – 1 | a(i) = b(j)}
```

For example, if `a` and `b` represent two sets of integers, this corresponds to the cardinality of the intersection.  Following the heuristic of replacing a constant in the postcondition, for symmetry, both `M` and `N` are replaced with variables:
```
x = #{i ∈ 0 .. m – 1, j ∈ 0 .. n – 1 | a(i) = b(j)}
```
However, there are multiple paths reaching `(m, n) = (M, N)` from `(m, n) = (0, 0)`. To obtain guidance, the invariant can be strengthened. Provided `a(m) ≠ b(n)`, index `m` can be incremented and `x` kept unchanged only if no coincidence is overlooked, meaning that all elements of `b[0..n)` are less than all elements of  `a[m..M)`. To be able to use this, the resulting invariant has, for symmetry, two such conditions conjoined:

```
Q(m, n) ≡ x = #{i ∈ 0 .. m – 1, j ∈ 0 .. n – 1 | a(i) = b(j)} ∧ b[0 .. n) < a[m .. M) ∧ a[0 .. m) < b[n .. N)
```

The desired postcondition is `Q(M, N)`, which follows already from `Q(m, n) ∧ (m = M ∨ n = N)`. Thus, the resulting loop structure is:

```algorithm
k, m, n := 0, 0, 0
{0 ≤ m ≤ M ∧ 0 ≤ n ≤ N ∧ Q(m, n)}
while m < M ∧ n < N do
    {0 ≤ m < M ∧ 0 ≤ n < N ∧ Q(m, n)}
    "modify k, m, n"
    {0 ≤ m ≤ M ∧ 0 ≤ n ≤ N ∧ Q(m, n)}
{P(N)}
```

To preserve `b[n .. N) > a[0 .. m)`, the statement `m := m + 1` must be guarded by `b(n) > a(m)`. The statement `k, m, n := k + 1, m + 1, n + 1` must be guarded by `a(m) = b(n)`. Thus, the complete program is:

```algorithm
k, m, n := 0, 0, 0
{0 ≤ m ≤ M ∧ 0 ≤ n ≤ N ∧ Q(m, n)}
while m < M ∧ n < N do
    {0 ≤ m < M ∧ 0 ≤ n < N ∧ Q(m, n)}
    if a(m) < b(n) → m := m + 1
    [] a(m) = b(n) → k, m, n := k + 1, m + 1, n + 1
    [] a(m) > b(n) → n := n + 1
    {0 ≤ m ≤ M ∧ 0 ≤ n ≤ N ∧ Q(m, n)}
{P(N)}
```



## Program Development: Printing Images

Larger programs are decomposed into _modules_ that may have _private_ variables which can only be accessed through _public_ procedures, a principle known as _encapsulation_. While the inner workings of a module may be intricate, the module's specification should remain simple.

The following development illustrates how a module is specified with _abstract variables_, which are used in annotations. The example also shows how to reason about program output.

The task is to print a picture given by the arrays `fx`, `fy`, where

```algorithm
∀ i ∈ 0 .. N – 1 • 0 ≤ fx(i) < X  ∧  0 ≤ fy(i) < Y
```
That means with each input of `i` from `0` to `N – 1`, two functions `fx(i)` and `fy(i)` indicate each printed point.   

We assume that the printer prints only one black dot at a time and starts at the upper left corner. The printer is controlled by the following commands:

- `newLine`: start a new line at the leftmost position
- `advance`: move the print head one position to the right
- `print`: print a dot and move one position to the right

The state of the printer is represented by the state of the paper and the coordinates of the print head. Each dot on the paper is represented by a boolean value, `true` for black and `false` for white.

```algorithm
var paper: 0 .. X – 1 × 0 .. Y – 1 → bool     // paper is a 2D array
var hx, hy: integer                                       // the position of the printer.
```

```algorithm
newLine:
    hx, hy := 0, hy + 1
advance:
    hx := hx + 1
print:
    paper(hx, hy) := true ; hx := hx + 1
```

Initially, the paper is blank, and the print head is at the origin:

```algorithm
P0: (∀ i ∈ 0 .. X – 1, j ∈ 0 .. Y – 1  •  ¬paper(i, j))   ∧   hx = 0 ∧ hy = 0
```
The printing task is then specified by:

```algorithm
{P1: P0 ∧ (∀ i ∈ 0 .. N – 1    •    0 ≤ fx(i) < X ∧ 0 ≤ fy(i) < Y)}
printPicture
{∀ i ∈ 0 .. X – 1, j ∈ 0 .. Y – 1   • 
    paper(i, j) = (∃ k ∈ 0 .. N – 1   •   i = fx(k)  ∧   j = fy(k))}
```

The approach is first to build the whole image in memory and then to print it. To this end, we declare a variable of the size of the paper:

```algorithm
printPicture:
    var image: 0 .. X – 1 × 0 .. Y – 1 → bool
        buildImage
        {P0 ∧ ∀ i ∈ 0 .. X – 1, j ∈ 0 .. Y – 1 •
            image(i, j) = (∃ k ∈ 0 .. N – 1 • i = fx(k) ∧ j = fy(k))}
        printImage
        {paper = image}
```

We refine `buildImage`: first, `image` is cleared, and then the dots are set:

```algorithm
buildImage:
    clearImage
    {P1 ∧ (∀ i ∈ 0 .. X – 1, j ∈ 0 .. Y – 1 • ¬image(i, j))}
    markDots
```

We refine `clearImage` by clearing all lines:

```algorithm
clearImage:
    var y : integer
        y := 0
        {P2: P1 ∧ (∀ i ∈ 0 .. X – 1, j ∈ 0 .. y – 1 • ¬image(i, j))}
        while y < Y do
            clearLine(y) ; y := y + 1
```

We refine `clearLine(y)` by clearing all dots of line `y`:

```algorithm
clearLine(y):
    var x : integer
        x := 0
        {P2 ∧ (∀ i ∈ 0 .. x – 1 • ¬image(i, y))}
        while x < X do
            image(x, y) := false ; x := x + 1
```

We refine `markDots` by iterating over all `N` dots to be drawn:

```algorithm
markDots:
    var n : integer
        n := 0
        {∀ x ∈ 0 .. X – 1, y ∈ 0 .. Y – 1 •
            image(x, y) = (∃ i ∈ 0 .. n – 1 • x = fx(i) ∧ y = fy(i))}
        while n < N do
            image(fx(n), fy(n)) := true ; n := n + 1
```

We refine `printImage` by printing all lines from top to bottom:

```algorithm
printImage:
    var y : integer
        y := 0
        {P3: hy = y ∧ ∀ i ∈ 0 .. X – 1, j ∈ 0 .. y – 1 •
            paper(i, j) = image(i, j)   ∧  
            paper(i, j) = (∃ k ∈ 0 .. N – 1  •  i = fx(k)  ∧   j = fy(k))}
        while y < Y do
            printLine(y) ; newLine ; y := y + 1
```

Finally we refine `printLine(y)` by printing all dots of a line:

```algorithm
printLine(y):
    var x : integer
        x := 0
        {P3  ∧  hx = x ∧ (∀ i ∈ 0 .. x – 1 •
            paper(i, y) = (∃ k ∈ 0 .. N – 1   •   i = fx(k)   ∧   y = fy(k)))}
        while x < X do
            if image(x, y) then print else advance
            x := x + 1
```

By putting all the parts together, we arrive at the complete program:

```algorithm
printPicture:
    var image: 0 .. X – 1 × 0 .. Y – 1 → bool      // image is a 2D array
    var x, y, n : integer

    y := 0
    while y < Y do        // clear the image
        x := 0
        while x < X do    // clear the image for each line
            image(x, y) := false ; x := x + 1
        y := y + 1
    
    n := 0
    while n < N do        // markdot to the image 
        image(fx(n), fy(n)) := true ; n := n + 1
    
    y := 0
    while y < N do        // print to the paper
        x := 0
        while x < X do    // print to the paper for each line
            if image(x, y) then print else advance
            x := x + 1
        newLine ; y := y + 1
```

The formal proof is laborious as intermediate annotations get long. For example, `P0` has to be "carried along" in all intermediate annotations of  `buildImage` to be available as a precondition of `printImage`.

The following two rules help to reason with smaller annotations:

**Rules for simplification of correctness reasoning**

<div style="display:table; border-top:1em solid transparent">
  <div style = "display:table-cell; border-left:24px solid transparent; vertical-align:middle">
    <code>{P} S {P}</code> <br><br>
    <code>{P0 ∧ P1} S {Q0 ∧ Q1}</code>
  </div>
  <div style = "display:table-cell; border-left:24px solid transparent; vertical-align:middle" >
    if variables assigned in <code>S</code> do not occur in <code>P</code> <br><br>
    if &nbsp; <code>{P0} S {Q0}</code> &nbsp; and &nbsp; <code>{P1} S {Q1}</code>
  </div>
</div>

## References